In [79]:
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
from matplotlib import cm
import seaborn as sns
from pathlib import Path
from sklearn.metrics import mean_squared_error

matplotlib.rcParams.update({'font.style': 'normal'})
matplotlib.rcParams.update({'font.sans-serif': 'Segoe UI'})
matplotlib.rcParams.update({'font.size': 26})

colors = sns.color_palette(palette='Accent').as_hex()

In [103]:
def solutions_different_sources(df1, df1name, df2, df2name, temp_threshold):
    merged_df = pd.merge(df1, df2, on=['solute_smiles', 'solvent_smiles'], suffixes=('_' +df1name, '_' +df2name))

    # Filter rows where temperature is within threshold
    merged_df['temp_diff'] = abs(merged_df['temperature_'+df1name] - merged_df['temperature_'+df2name])
    merged_df = merged_df[merged_df['temp_diff'] <= temp_threshold]
    merged_df = merged_df.loc[merged_df.groupby(['solute_smiles', 'solvent_smiles'])['temp_diff'].idxmin()]
    solutions_with_different_sources = merged_df[~merged_df.apply(lambda row: row['source_'+df1name] in row['source_'+df2name], axis=1)]

    RMSE = np.sqrt(mean_squared_error(solutions_with_different_sources["logS_"+df1name], solutions_with_different_sources["logS_"+df2name]))
    return RMSE, solutions_with_different_sources

In [104]:
_src_dir: str = os.path.join(r"C:\Users\ChemeGrad2021\Documents\highsol\data")
bigsol: pd.DataFrame = pd.read_csv(os.path.join(_src_dir, "krasnov\BigSolDB.csv"))
solprop: pd.DataFrame = pd.read_csv(os.path.join(_src_dir, "SolProp_v1.2\Data\CombiSolu-Exp.csv"))

leeds_acetone: pd.DataFrame = pd.read_csv(os.path.join(_src_dir, "leeds/acetone_solubility_data.csv"))
leeds_acetone['solvent_smiles'] = 'CC(=O)C'

leeds_benzene: pd.DataFrame = pd.read_csv(os.path.join(_src_dir, "leeds/benzene_solubility_data.csv"))
leeds_benzene['solvent_smiles'] = 'C1=CC=CC=C1'


leeds_ethanol: pd.DataFrame = pd.read_csv(os.path.join(_src_dir, "leeds/ethanol_solubility_data.csv"))
leeds_ethanol['solvent_smiles'] = 'CCO'

leeds = pd.concat([leeds_acetone, leeds_benzene, leeds_ethanol], ignore_index=True, axis=0) 

In [105]:
solprop = solprop.rename(columns={'experimental_logS [mol/L]': 'logS_solprop'})
leeds = leeds.rename(columns={'SMILES': 'solute_smiles'})
leeds = leeds.rename(columns={'Source': 'source'})
leeds = leeds.rename(columns={'T': 'temperature'})
leeds = leeds.rename(columns={'LogS': 'logS_leeds'})
leeds['temperature'] = leeds['temperature'] + 273.15

bigsol = bigsol.rename(columns={'SMILES': 'solute_smiles'})
bigsol = bigsol.rename(columns={'SMILES_Solvent': 'solvent_smiles'})
bigsol = bigsol.rename(columns={'Source': 'source'})
bigsol = bigsol.rename(columns={'T,K': 'temperature'})
bigsol['Solubility'] = np.log10(bigsol['Solubility'])
bigsol = bigsol.rename(columns={'Solubility': 'logS_bigsol'})

In [131]:
RMSE, solutions_with_different_sources_solprop_bigsol = solutions_different_sources(solprop, 'solprop', bigsol, 'bigsol', 2)
print(RMSE)

1.2902410567336358


Leeds and Solprop

In [136]:
RMSE, solutions_with_different_sources_solprop_leeds = solutions_different_sources(solprop, 'solprop', leeds, 'leeds', 2)
print(RMSE)

0.0006992831709697889


Leeds and Bigsol

In [137]:
RMSE, solutions_with_different_sources_bigsol_leeds = solutions_different_sources(bigsol, 'bigsol', leeds, 'leeds', 2)
print(RMSE)

1.2286580058987124


Now, we create a dataframe that contains all of the overlapping solutions with non-overlapping sources in a single dataset. 

In [138]:
merged_dfs = [
    (solutions_with_different_sources_bigsol_leeds, 'bigsol', 'leeds'),
    (solutions_with_different_sources_solprop_bigsol, 'solprop', 'bigsol'),
    (solutions_with_different_sources_solprop_leeds, 'solprop', 'leeds')
]

final_dfs = []

for df, name1, name2 in merged_dfs:
    df_final = df[['solute_smiles', 'solvent_smiles', 'temp_diff',
                   f'logS_{name1}', f'logS_{name2}',
                   f'temperature_{name1}', f'temperature_{name2}',
                   f'source_{name1}', f'source_{name2}']].copy()
    
    # Rename columns to standardized names
    df_final = df_final.rename(columns={
        f'logS_{name1}': 'logS_1',
        f'logS_{name2}': 'logS_2',
        f'temperature_{name1}': 'temperature_1',
        f'temperature_{name2}': 'temperature_2',
        f'source_{name1}': 'source_1',
        f'source_{name2}': 'source_2'
    })
    
    df_final['dataset1'] = name1
    df_final['dataset2'] = name2
    
    final_dfs.append(df_final)

# Concatenate all merged dataframes
final_df = pd.concat(final_dfs, ignore_index=True)
final_df

,solute_smiles,solvent_smiles,temp_diff,logS_1,logS_2,temperature_1,temperature_2,source_1,source_2,dataset1,dataset2
0,C1CN2CCN1CC2,CCO,0.01,-0.453457,-0.798443,298.15,298.14,10.1021/je8005375,Reaxys,bigsol,leeds
1,C1N2CN3CN1CN(C2)C3,CCO,0.34,-2.017729,-0.788571,293.49,293.15,10.1021/je800662m,Reaxys,bigsol,leeds
2,CC(=O)NCC(=O)O,CCO,1.01,-2.303906,-1.086940,298.15,297.14,10.1021/acs.jced.0c00983,Reaxys,bigsol,leeds
3,CC(C)(O)P1(=O)OCC2(CO1)COP(=O)(C(C)(C)O)OC2,CCO,0.01,-2.734710,-1.501169,297.63,297.62,10.1021/je700050r,Reaxys,bigsol,leeds
4,CC1(C)COP(=O)(C(C)(C)O)OC1,CCO,0.01,-1.457175,-0.223633,298.35,298.34,10.1021/je4003114,Reaxys,bigsol,leeds
5,CCCCCCCCCCCC(=O)O,CCO,0.99,-0.712646,0.555510,294.14,293.15,10.1021/je500222s,Reaxys,bigsol,leeds
6,CCCCCCCCCCCCCCCCCC(=O)O,CCO,1.70,-2.244125,-0.600166,299.85,298.15,10.1021/je7006567,Reaxys,bigsol,leeds
7,COc1cnc(Cl)nc1Cl,CCO,0.01,-1.323947,-0.090406,299.44,299.43,10.1021/je9005689,Reaxys,bigsol,leeds
8,c1ccc2cc3ccccc3cc2c1,CCO,0.00,-3.368556,-2.103701,298.15,298.15,10.1021/acs.jced.8b00536,Reaxys,bigsol,leeds
9,CC(=O)N1CCC[C@H]1C(=O)O,C1COCCO1,0.00,-0.541413,-1.624519,288.15,288.15,https://doi.org/10.1021/acs.jced.1c00032,10.1021/acs.jced.1c00032,solprop,bigsol


In [141]:
unique_solvents = final_df['solvent_smiles'].nunique()
unique_solutes = final_df['solute_smiles'].nunique()

print("There are " +str(len(final_df)) + " solutions with different sources, containing " + str(unique_solutes) + " unique solutes, and " + str(unique_solvents) + " unique solvents.")

RMSE = np.sqrt(mean_squared_error(final_df["logS_1"], final_df["logS_2"]))
print("Overall inter-laboratory experimental variability has RMSE = " + f"{RMSE}")

There are 42 solutions with different sources, containing 11 unique solutes, and 11 unique solvents.
Overall inter-laboratory experimental variability has RMSE = 1.2616835733740859
